In [ ]:
import random

%matplotlib inline
import networkx as nx

# Lab 7

This notebook is based on Chapter 6 tutorial by F. Menczer, S. Forunato, C.A. Davis, *A First Course in Network Science* (Cambridge 2020), available at: https://cambridgeuniversitypress.github.io/FirstCourseNetworkScience/.
The contents of the tutorials have been adapted by KAK.

Contents:

1. Tutorials

    1.1 Partitions

    1.2. Modularity

    1.3. Zachary's Karate Club

    1.4. Girvan-Newman clustering algorithm

2. Exercises

    2.1 Exercise 1: Girvan-Newman algorithm

    2.2 Exercise 2: Kernighan-Lin algorithm

    2.3 Exercise 3: Modularity in Gephi

## 1. Tutorials

### 1.1 Partitions

During the lecture we've learned that a **partition** of a graph is a separation of its nodes into disjoint groups. 
Formally, a partition is a list of sets such that every node is in exactly one set.

In Network Science there are two partitions which are referred to as the trivial partitions:

1. The partition with one set containing every node (Singleton Partition).
2. The partition with N sets, each containing a single node (Partition of Singletons).

A valid partition thus contains between 1 and N sets.  

Consider the following graph representing two separate groups of students who interact with each other internally on regular basis (as they are in the same study programme and take the same classes), but never interact with the other group. 
Let one group be TNAH and the other HN.  

Let's assume that in each group there is one student who is involved in the Student Enterprise of École nationale des Chartes, CartaData. Let is be Ann for TNAH and Zack for HN.  Ann and Zack interact with each other within CartaData creating a link between the two student groups.

This can be visualised as follows: 

In [ ]:
G = nx.Graph()
nx.add_cycle(G, ["Ann", "Bob", "Camilla", "Dave"])
nx.add_cycle(G, ["Uma", "Yige", "Walter", "Zack"])
G.add_edge("Ann", "Zack")

nx.draw(G, with_labels=True)

Now let's think that we would like to identify the communities within our network. How would you like to analyse and identify the relationships between the students? 

You could partition the nodes into three groups in the following way: 

In [ ]:
partition = [
    {"Bob", "Camilla", "Dave"},
    {"Uma", "Yige", "Walter"},
    {"Ann", "Zack"},
]
print(partition)

Remember that every node in the graph can be in exactly one of the sets in the partition, so if you want to somehow highlight the CartaData relationship, you have to put Ann and Zack into a separate bag.

Using NetworkX and its function `community.is_partition` we can verify that our partition is valid (i.e. one partition per node):

In [ ]:
nx.community.is_partition(G, partition)

When developing community detection algorithms, we often make use of a *partition map*, which is a dictionary mapping node names to a partition index. This is useful for quickly comparing if two nodes are in the same cluster in the partition:

In [ ]:
partition_map = {}
for idx, cluster_nodes in enumerate(partition):
    for node in cluster_nodes:
        partition_map[node] = idx

partition_map

In this dictionary, the keys are the node names and two nodes will have the same value if they are in the same partition.
Let's test  (expecting only True or False answer) whether "Ann" and "Bob" are in the same partition: 

In [ ]:
partition_map["Ann"] == partition_map["Bob"]

We can visualize our partition by drawing the graph with nodes colored by their partition membership:

In [ ]:
node_colors = [partition_map[n] for n in G.nodes]
        
nx.draw(G, node_color=node_colors, with_labels=True)

Ok, but what if we don't want to do this manually (and we never do)? 

We've learned during the lecture that there are various methods of partitioning our graph; one of them is bisection (i.e.
we want to partition a graph into two blocks). 

We can use the Kernighan–Lin algorithm, which is included in the NetworkX function: `kernighan_lin_bisection` (G, partition=None, max_iter=10, weight='weight', seed=None), described here: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.kernighan_lin.kernighan_lin_bisection.html

Let's use it on our graph. 




In [ ]:
new_partition = list(nx.community.kernighan_lin_bisection(G))
print(new_partition)

Now:

In [ ]:
new_partition_map = {}
for idx, cluster_nodes in enumerate(new_partition):
    for node in cluster_nodes:
        new_partition_map[node] = idx
print("New partition:", new_partition_map)
print("Is partitionL", nx.community.is_partition(G, new_partition))

new_node_colors = [new_partition_map[n] for n in G.nodes]
nx.draw(G, node_color=new_node_colors, with_labels=True)

So by bisecting our graph (dividing the graph into two partitions), we are able to distinguish two groups of nodes corresponding to TNAH & HN students. However, we have no way of finding out that Ann and Zack worked together in CartaData.  

Think about the question how we could represent the existence of CartaData? 

Note your answer below before you move to the next section .  

_Answer_

### 1.2. Modularity

At a high level, network community detection consists of finding a partition that achieves good separation between the groups of nodes. Before we get into how to find good partitions of a graph, we need an objective -- a way to measure how good the partition is. Modularity is one such objective function.

The modularity of a graph partition compares the number of intra-group edges with a random baseline. Higher modularity scores correspond to a higher proportion of intra-group edges, therefore fewer inter-group edges and better separation of groups.

Box 6.1 in the textbook describes how modularity is calculated. Here, let's try to calculate modularity for weighted undirected networks (Equation 6.6). It is calculated using the total weight of the links of the network `W`, the total weight of the internal links of a given cluster `W_c`, and the total strength of the nodes of the cluster `s_c`.

In [ ]:
# We will define here our own function to calculate modularity. 
def my_modularity(G, part):
    W = sum(G.edges[v, w].get('weight', 1) for v, w in G.edges) # sum up all edge weights in the graph (if an edge has no weight attribute, assume weight of 1)
    summation = 0
    for cluster_nodes in part: # Iterate through each community in the partition.
        s_c = sum(G.degree(n, weight='weight') for n in cluster_nodes) # Compute the sum of weighted degrees (strengths) in the community
        C = G.subgraph(cluster_nodes)   # Use subgraph to count only internal links
        #Compute the total internal edge weight in the community 
        W_c = sum(C.edges[v, w].get('weight', 1) for v, w in C.edges)
        summation += W_c - s_c ** 2 / (4 * W)
    return summation / W

part = new_partition # You can change between `partition` and `new_partition` defined above to see how the modularity changes and consider which one is better? 
my_modularity(G, part)

The values we get as a result of this calculation should be 0.38 for the partition we got through bisection (dividing our nodes into two groups of nodes, `new_partition`), and 0.22 for the partition we manually introduced (dividing our network into three groups of nodes `partition`). 



### NetworkX function

NetworkX provides a modularity function `community.quality.modularity` that is more efficient than ours, but its result will be the same as ours. 

Test it for both scenarios.

In [ ]:
nx.community.quality.modularity(G, new_partition)

*Remember:* 

Higher modularity (~0.3 to 0.7) → Stronger community structure.

Lower modularity (~0.0 to 0.1) → Weak or no community structure.

Negative modularity → Worse than random clustering.

### 1.3. Zachary's Karate Club

When writing and testing community-detection algorithms, it helps to make use of benchmark networks: graphs with a known, "natural" community structure. Perhaps the most famous benchmark graph is Zachary's Karate Club. It contains 34 nodes, representing members of a karate club whose interactions were monitored over a period of three years by researchers. Links in this graph connect individuals interacting outside club activities, a proxy for social ties.

During the course of the study, a conflict between the instructor Mr. Hi (node 0) and the president, or Officer (node 33) led to a split of the club into separate groups led by Mr. Hi and Officer. In this case we know whom each member of the group followed after the split, providing empirical community labels: those members who followed Mr. Hi are said to be one community and those following the Officer make up the other.

For this graph, we assume that the post-split group composition was largely driven by the social ties: members of the same friend groups would want to be part of the same club after the split. We thus expect a good community-detection algorithm to predict the post-split group composition with high accuracy.

Zachary's karate club is such a popular benchmark graph that it has its own function in NetworkX:

In [ ]:
K = nx.karate_club_graph()
nx.draw(K, with_labels=True)

Each node in a NetworkX graph has a dictionary of *attributes* associated with it. This dictionary can hold arbitrary data about a node. We can get the attributes for a single node by giving the node name to the `nodes` object.

Each node in this graph has a `'club'` attribute, indicating whether the member followed the instructor or the president after the split:

In [ ]:
K.nodes[0]

In [ ]:
K.nodes[9]

We can visualize these labels by coloring each node according to its `'club'` attribute:

In [ ]:
K = nx.karate_club_graph()
club_color = {
    'Mr. Hi': 'orange',
    'Officer': 'lightblue',
}
node_colors = [club_color[K.nodes[n]['club']] for n in K.nodes]
nx.draw(K, node_color=node_colors, with_labels=True)

This separation looks good, in that there are relatively few inter-community links as opposed to intra-community links. Let's create a graph partition based on these labels and measure its modularity.

We can do this by creating a dictionary of two sets, one for each value of the nodes' `'club'` attribute, then assigning the nodes to the corresponding set. 

In [ ]:
groups = {
    'Mr. Hi': set(),
    'Officer': set(),
}

for n in K.nodes:
    club = K.nodes[n]['club']
    groups[club].add(n)
    
groups

By using the dictionary's `.values()` method, we can get a list of sets that define our partition:

In [ ]:
empirical_partition = list(groups.values())
empirical_partition

In [ ]:
nx.community.is_partition(K, empirical_partition)

Since our partition is indeed a valid partition, we can get the modularity of this partition:

In [ ]:
nx.community.quality.modularity(K, empirical_partition)

This is a relatively high modularity, which is what we expect.

### Comparison to a random partition

For the sake of comparison, let's generate a random partition of this network and check its modularity. We would expect a modularity close to zero in this case.

First we generate a sample of 17 nodes, half the total number of nodes, and assign them to one community. Our second community then includes the nodes in the graph not in the first community. We can use some set arithmetic to do this concisely:

In [ ]:
random_nodes = random.sample(list(K.nodes), 17)
#Create a list of two sets
# The first set contains the 17 randomly selected nodes.
# The second set contains all other nodes in the graph.
random_partition = [set(random_nodes),
                    set(K.nodes) - set(random_nodes)] 
random_partition

We can visualize this partition and observe that the communities are much less natural-looking, as we would expect from a random assignment.

In [ ]:
random_node_colors = ['orange' if n in random_nodes else 'lightblue' for n in K.nodes]
nx.draw(K, node_color=random_node_colors)

And finally we can test the modularity of this partition:

In [ ]:
nx.community.quality.modularity(K, random_partition)

Since this is a random process the modularity won't be exactly zero, but it should be fairly close. Go ahead and repeat the process of generating a random partition and testing its modularity -- it will fluctuate around its mean value of zero.

### 1.4. Girvan-Newman clustering

Our task in this part will be to implement the Girvan-Newman clustering algorithm. Since NetworkX can do the heavy lifting for us -- computing betweenness centrality -- the code part of the task is relatively straightforward. Most of our effort here is spent interpreting and explaining intermediate results.

Let's recall from the lecture (and/or textbook) that the Girvan-Newman clustering algorithm helps us to:

1. Create a partition sequence
  1. Calculate the betweenness centrality for all links.
  2. Remove the link with largest betweenness and create a partition using connected components.
  3. Recalculate the betweenness centrality of the links of the resulting graph.
  4. Repeat from step two until no links remain.
2. Evaluate each partition in the sequence and choose the one with the highest modularity.

During this process, the number of connected components in the graph will increase monotonically as clusters are broken up. Since we are removing one link at a time, the number of connected components can increase by at most one between steps in the sequence -- it's not possible for a single edge to connect more than two nodes, and thus components.

We hope that the resulting partition of the graph will approximate its underlying community structure. We'll use the Karate Club graph here because we know the ground-truth community labels and can compare the result obtained from the algorithm.

In [ ]:
G = nx.karate_club_graph()
nx.draw(G)

#### 1.4.1 Create a partition sequence
##### 1.4.1.1. Calculate the betweenness centrality for all links

NetworkX does the heavy lifting here. All we need to do is understand the output.

In [ ]:
nx.edge_betweenness_centrality(G)

The resulting dictionary has edge tuples as the keys, and each associated value is the betweenness centrality of that edge. The algorithm to compute the edge betweenness of all edges in a graph costs about the same as calculating it for a single edge, so we'll make use of this dictionary with the computed values for every edge.

Once computed for all edges, we can easily get the associated betweenness for a single edge. For example, to get the edge betweenness of the edge between nodes 0 and 1:

In [ ]:
my_edge_betweenness = nx.edge_betweenness_centrality(G)
my_edge_betweenness[0, 1]

Recall that dictionaries also have the `.get` method. This will be used in the next step.

In [ ]:
my_edge_betweenness.get((0, 1))

##### 1.4.1.2. Remove the link with largest betweenness...

Given this dictionary of betweenness values for each edge, we can make use of Python's inbuilt `max` function to give us the key in this dictionary with the greatest value. Since there is a key in the dictionary for each edge in the graph, the following two expressions are equivalent, but the second one is probably more explicit as to what we're doing with this statement.

I'm using the name `my_edge_betweenness` to make clear that this is a dictionary we've named and not a NetworkX function.

In [ ]:
max(my_edge_betweenness, key=my_edge_betweenness.get)

In [ ]:
max(G.edges(), key=my_edge_betweenness.get)

This is then the edge we want to remove at this step in the process:

In [ ]:
my_edge_betweenness = nx.edge_betweenness_centrality(G)
most_valuable_edge = max(G.edges(), key=my_edge_betweenness.get)
print("The most valuable edge (with the highest betweenness centrality) is:",  most_valuable_edge)
print("Unpacked:",*most_valuable_edge)
G.remove_edge(*most_valuable_edge)

The "splat" (`*`-operator) in the last statement above `G.remove_edge(*most_valuable_edge)` performs tuple unpacking into the arguments of a function. For example, if our most valuable edge is `(0, 31)`,

    G.remove_edge(*most_valuable_edge)
is the same as
    
    G.remove_edge(most_valuable_edge[0], most_valuable_edge[1])
or

    G.remove_edge(0, 31)

#### ...and create a partition using connected components

This is almost a too simple example, because the `nx.connected_components()` function gives us almost exactly what we want: 

In [ ]:
nx.connected_components(G)

We just have to remember to ask for it in a list:

In [ ]:
list(nx.connected_components(G))

Remember: a partition is a list of sets where every node is in exactly one of these sets. This is just what we have here, although it's a bit boring since we've only removed one edge and so there is still one connected component. If you like, you can try running the previous two cells a few times until you have more than one connected component so you can see what that looks like.

Note that this feature whereby the connected components correspond exactly to our putative community labels is particular to the Girvan-Newman algorithm: other clustering algorithms may use different ways of generating their partitions.

##### 1.4.1.3. Recalculate the betweenness centrality of the links of the resulting graph.
##### 1.4.1.4. Repeat from step two until no links remain.

This implies that we need a loop to repeat this process $L$ times, once for each edge, and that we should keep track of the partitions generated. Straightforward stuff. We'll start with a fresh Karate Club graph since we removed some edges above:

In [ ]:
G = nx.karate_club_graph()
partition_sequence = []
for _ in range(G.number_of_edges()):
    my_edge_betweenness = nx.edge_betweenness_centrality(G)
    most_valuable_edge = max(G.edges(), key=my_edge_betweenness.get)
    G.remove_edge(*most_valuable_edge)
    my_partition = list(nx.connected_components(G))
    partition_sequence.append(my_partition)

Note the idiomatic construction of this `for` loop. Using `_` as the name for the loop variable tells the reader that we don't expect to do anything with the loop variable -- we just want to perform a task a specific number of times. One might be tempted to use a `while` loop here, but that way lie dragons: a mistake in a `while` loop can lead to infinite loops which are a headache.

If we've done this right, we should have a partition for each step of the process, *i.e.* one for each edge in the graph:

In [ ]:
len(partition_sequence), nx.karate_club_graph().number_of_edges()

Since we started with a connected graph, removing one edge probably doesn't disconnect the graph, so our first partition probably only has one community:

In [ ]:
len(partition_sequence[0])

...and the last partition should also be trivial, with each node in its own community:

In [ ]:
len(partition_sequence[-1]), nx.karate_club_graph().number_of_nodes()

#### 1.4.2 Evaluate the modularity of each partition in the sequence

We now have a sequence of partitions and a function to calculate the modularity of a partition. This is a great time to use a list comprehension!
If you need a reminder, check out W3schools tutorial: https://www.w3schools.com/python/python_lists_comprehension.asp 


In [ ]:
G = nx.karate_club_graph()
modularity_sequence = [my_modularity(G, p) for p in partition_sequence] # Here we are calling out own function `my_modularity` we defined above 
modularity_sequence

This sequence is then the modularity of the partition at each step in the algorithm. The first several entries in this sequence are effectively zero while there is only one community/component, then it jumps up once there is more than one community. We can use pyplot to visualize this relationship:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(modularity_sequence)
plt.ylabel('Modularity')
plt.xlabel('Algorithm step')

### Get the partition with highest modularity

Visually, we see a peak in the modularity sequence. This is the partition that maximizes modularity, and thus the output of the algorithm. We can use the `max` function to get the partition with highest modularity. Ideally we want to write the following:

In [ ]:
best_partition = max(partition_sequence, key=nx.community.quality.modularity) # This will give us an error

...but we receive an error. Recall that a key function must take exactly one argument, the item in the sequence being evaluated, but the modularity function takes two arguments: the graph and the partition. We can fix this by writing a single-argument function to use as the key:

In [ ]:
def my_modularity1(partition):
    return nx.community.quality.modularity(G, partition)
best_partition = max(partition_sequence, key=my_modularity1)

Advanced Pythonauts will see a differet solution to this using the `zip` function to align the previously-generated partition & modularity sequences, but this solution is more explicit.

So after all that work, what is the best partition?

In [ ]:
best_partition

Interesting! The partition of the karate club graph with highest modularity actually has five components! Let's visualize them, using our code for partition maps we wrote back at the beginning of this tutorial:

In [ ]:
def create_partition_map(partition):
    partition_map = {}
    for idx, cluster_nodes in enumerate(partition):
        for node in cluster_nodes:
            partition_map[node] = idx
    return partition_map

In [ ]:
best_partition_map = create_partition_map(best_partition)

node_colors = [best_partition_map[n] for n in G.nodes()]
nx.draw(G, with_labels=True, node_color=node_colors)

Exactly how good is this five-community clustering?

In [ ]:
nx.community.quality.modularity(G, best_partition)

It's higher than the "ground truth" communities we evaluated in section 3, which is a good sign, but for the specific problem of trying to predict the post-split community membership, a clustering into five groups is useless to us.

### Get the best partition with a given number of communities

One of the most useful parts of the Girvan-Newman algorithm is that it is also useful when we have a specific number of clusters we want. In this case, we know the karate club split into two groups, so let's get the partition in the sequence with two components:

In [ ]:
for partition in partition_sequence:
    if len(partition) == 2:
        two_cluster_partition = partition
        break

two_cluster_partition

In [ ]:
two_cluster_partition_map = create_partition_map(two_cluster_partition)

node_colors = [two_cluster_partition_map[n] for n in G.nodes()]
nx.draw(G, with_labels=True, node_color=node_colors)

How good is this partition? We can get its modularity:

In [ ]:
nx.community.quality.modularity(G, two_cluster_partition)

Pretty good -- comparable to the ground truth community labels. Let's compare these side-by-side:

In [ ]:
import matplotlib.pyplot as plt # Remember for this to run, you need matplotlib installed in the virtual environment in which you're running your NA notebooks

pos = nx.layout.spring_layout(G)
fig = plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
two_cluster_partition_map = create_partition_map(two_cluster_partition)
node_colors = [two_cluster_partition_map[n] for n in G.nodes()]
nx.draw(G, with_labels=True, node_color=node_colors, pos=pos)
plt.title('Predicted communities')

plt.subplot(1, 2, 2)
node_colors = [G.nodes[n]['club'] == 'Officer' for n in G.nodes()]
nx.draw(G, with_labels=True, node_color=node_colors, pos=pos)
plt.title('Actual communities')

We can see that the predicted community labels are pretty accurate, only differing on a couple nodes that, visually, seem like they could plausibly belong to either group. 

Zachary's original paper even explains the practical considerations of one of these mispredicted nodes: node 8. 

Node 8 (or individual 9 in the original paper) was a supporter of the officer, but this individual was very near receiving a black belt from Mr. Hi and therefore did not want to leave the group. 

Read the original paper of Zachary (1977) here: https://www.researchgate.net/publication/248519014_An_Information_Flow_Model_for_Conflict_and_Fission_in_Small_Groups1/link/55428c610cf234bdb21a1658/download?_tp=eyJjb250ZXh0Ijp7ImZpcnN0UGFnZSI6InB1YmxpY2F0aW9uIiwicGFnZSI6InB1YmxpY2F0aW9uIn19



In [ ]:
G.nodes[8]

#### NetworkX Function

An attentive reader might note that there may be several bipartitions in the partition sequence we generated. 

We assert the following to be true:

1. For every integer 1 to N, the number of nodes, there is a partition in the sequence with that number of clusters
2. Every partition in the sequence with the same number of clusters is the same

As a consequence of these being true, optimized implementations of Girvan-Newman clustering will only store one partition for each number of clusters. This is how the implementation in NetworkX works, only providing one partition for each number of communities greater than one.

`nx.community.girvan_newman(G)` will generate a sequence containing one partition of each size greater than one. 

Below we can see the first five, and they are the same as those we generated using our own modularity function:

In [ ]:
list(nx.community.girvan_newman(G))[:5]

## 2. Exercises 
### Exercise 2.1: Girvan-Newman clustering

How would you check whether indeed all bipartitions in the list of all possible partitions we created with my_modularity are the same (our `partition_sequence`)? 

Write a code that checks how many bipartitions are there and then checks whether or not they are identical.

### Exercise 2.2: Kernighan-Lin algorithm 
Find the best bisection of Zachary's karate club network by applying Kernighan-Lin algorithm. 
You can use the `kernighan_lin_bisection()` of NetworkX. 

What is the modularity of this partition? 

Generate two plots, one with the benchmark and one with the Kernighan clustering. 
Compare the resulting from this bipartition with the benchmark (the natural split that took place in the karate network).

What are the differences? 

What does they tell us about the clustering algorithms in social networks? 

What do it tell us about the nodes in question? 

### Exercise 2.3: Modularity in Gephi

Export the network data in the format of your choice and import it into Gephi. Test how the modularity function works there. 
Does it use any of the functions we talked about today? 
What are the differences? 